# A notebook for creating sc graph animations with glimpses

In [1]:
import napari
from cellpose import models
from octopuslite import utils, tile
import numpy as np
import os

def view(img):
    return napari.Viewer().add_image(img)
from napari_animation import Animation
from tqdm.auto import tqdm

import btrack
import dask.array as da

from skimage.transform import rescale, resize, downscale_local_mean

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_palette("Spectral")

def msd_calc(x1, y1, x2, y2):
    return np.sqrt((x1-x2)**2+(y1-y2)**2)

### Load all metadata

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = utils.read_harmony_metadata(metadata_path, assay_layout=True)
assay_layout

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Iteratively load all tracks

In [3]:
tracks_dict = dict()
N_cell_t0_dict = dict()
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):

    with btrack.dataio.HDF5FileHandler(
                                        f"/mnt/DATA/macrohet/segmentation/tracks_objs/({row},{column})_tracks_rescaled.h5", 
                                        'r', 
                                        obj_type = 'obj_type_1', 
                                        ) as hdf: 
                                        tracks = hdf.tracks
                                        objs = hdf.objects
    N_cell_t0_dict[(row, column)] = len([obj for obj in objs if obj.t == 0])
    tracks_dict[(row, column)] = tracks

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/03/02 09:05:36 AM] Opening HDF file: /mnt/DATA/macrohet/segmentation/tracks_objs/(3,4)_tracks_rescaled.h5...
[INFO][2023/03/02 09:05:36 AM] Loading tracks/obj_type_1
[INFO][2023/03/02 09:05:36 AM] Loading LBEP/obj_type_1
[INFO][2023/03/02 09:05:36 AM] Loading objects/obj_type_1 (29079, 5) (29079 filtered: None)
[INFO][2023/03/02 09:05:37 AM] Loading objects/obj_type_1 (29079, 5) (29079 filtered: None)
[INFO][2023/03/02 09:05:37 AM] Closing HDF file: /mnt/DATA/macrohet/segmentation/tracks_objs/(3,4)_tracks_rescaled.h5
[INFO][2023/03/02 09:05:37 AM] Opening HDF file: /mnt/DATA/macrohet/segmentation/tracks_objs/(3,5)_tracks_rescaled.h5...
[INFO][2023/03/02 09:05:37 AM] Loading tracks/obj_type_1
[INFO][2023/03/02 09:05:37 AM] Loading LBEP/obj_type_1
[INFO][2023/03/02 09:05:37 AM] Loading objects/obj_type_1 (30240, 5) (30240 filtered: None)
[INFO][2023/03/02 09:05:37 AM] Loading objects/obj_type_1 (30240, 5) (30240 filtered: None)
[INFO][2023/03/02 09:05:38 AM] Closing HDF file:

[INFO][2023/03/02 09:05:56 AM] Loading LBEP/obj_type_1
[INFO][2023/03/02 09:05:56 AM] Loading objects/obj_type_1 (32090, 5) (32090 filtered: None)
[INFO][2023/03/02 09:05:56 AM] Loading objects/obj_type_1 (32090, 5) (32090 filtered: None)
[INFO][2023/03/02 09:05:58 AM] Closing HDF file: /mnt/DATA/macrohet/segmentation/tracks_objs/(5,7)_tracks_rescaled.h5
[INFO][2023/03/02 09:05:58 AM] Opening HDF file: /mnt/DATA/macrohet/segmentation/tracks_objs/(5,8)_tracks_rescaled.h5...
[INFO][2023/03/02 09:05:58 AM] Loading tracks/obj_type_1
[INFO][2023/03/02 09:05:58 AM] Loading LBEP/obj_type_1
[INFO][2023/03/02 09:05:58 AM] Loading objects/obj_type_1 (30332, 5) (30332 filtered: None)
[INFO][2023/03/02 09:05:59 AM] Loading objects/obj_type_1 (30332, 5) (30332 filtered: None)
[INFO][2023/03/02 09:05:59 AM] Closing HDF file: /mnt/DATA/macrohet/segmentation/tracks_objs/(5,8)_tracks_rescaled.h5
[INFO][2023/03/02 09:05:59 AM] Opening HDF file: /mnt/DATA/macrohet/segmentation/tracks_objs/(5,9)_tracks_re

# Fully tracked cells

In [4]:
tracks_df = pd.DataFrame(columns = ['Time', 'Intracellular Mtb content', 'Strain', 'Compound', 'Concentration'])

# Creating glimpses

Need to redefine the df to include xy coords

In [5]:
dfs = list()
filtered_tracks = dict()
for key in tracks_dict.keys():
    filtered_tracks[key] = [track for track in tracks_dict[key] if len(track) == 75]
    for track in filtered_tracks[key]:
        info = assay_layout.loc[key]
        strain = info['Strain']
        comp = info['Compound']
        conc = info['ConcentrationEC']
        d = {'Time (hours)':track['t'], 'x':track['x'],
             'y':track['y'],'Area':track['area'], 
     'Intracellular Mtb content':track['mean_intensity-1'],
     'Macroph. GFP expression':track['mean_intensity-0'],
     'Eccentricity':np.sqrt(1-((track['minor_axis_length']**2)/(track['major_axis_length']**2))),
     'MSD': [msd_calc(track['x'][i-1], track['y'][i-1], 
                      track['x'][i], track['y'][i]) 
               if i != 0 else 0
             for i in range(0, len(track))],
     'Strain':[strain for i in range(len(track['t']))], 
     'Compound':[comp for i in range(len(track['t']))], 
     'Concentration':[conc for i in range(len(track['t']))], 
     'Cell ID':[track.ID for i in range(len(track['t']))],
     'Acquisition ID':[key for i in range(len(track['t']))]}
        dfs.append(pd.DataFrame(d))
df = pd.concat(dfs, ignore_index=True)

In [8]:
df

,Time (hours),x,y,Area,Intracellular Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID
0,0,1169.847412,13.893421,760.0,0.005136,0.014705,0.809516,0.000000,RD1,CTRL,EC0,343,"(3, 4)"
1,1,1172.826538,14.674929,1049.0,0.005121,0.013235,0.843927,3.079926,RD1,CTRL,EC0,343,"(3, 4)"
2,2,1174.588867,14.375803,934.0,0.005116,0.013899,0.719524,1.787535,RD1,CTRL,EC0,343,"(3, 4)"
3,3,1175.250610,12.160207,774.0,0.005094,0.013428,0.739700,2.312308,RD1,CTRL,EC0,343,"(3, 4)"
4,4,1180.553955,9.897770,538.0,0.005071,0.012774,0.761300,5.765768,RD1,CTRL,EC0,343,"(3, 4)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81520,70,559.663025,1177.240234,3018.0,0.005635,0.024949,0.698306,3.245095,WT,BDQ,EC99,155,"(6, 9)"
81521,71,556.457581,1178.115601,2863.0,0.005609,0.024090,0.716728,3.322821,WT,BDQ,EC99,155,"(6, 9)"
81522,72,559.905823,1178.514648,3047.0,0.005600,0.022092,0.789115,3.471255,WT,BDQ,EC99,155,"(6, 9)"
81523,73,565.014221,1180.512939,2737.0,0.005606,0.021206,0.822009,5.485335,WT,BDQ,EC99,155,"(6, 9)"


In [9]:
assay_layout

Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

In [6]:
## picking EC99 PZA as example 
tracks_df = df[df['Acquisition ID']==(3, 5)]

#### pull corresponding images

In [7]:
row, column = tracks_df['Acquisition ID'].iloc[0]
images = tile.compile_mosaic(
                             image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_plane = 'sum_proj',
                             ).astype(np.uint16)

In [12]:
images

,Array,Chunk
Bytes,10.22 GiB,15.50 MiB
Shape,"(75, 2, 6048, 6048)","(1, 2, 2016, 2016)"
Count,16875 Tasks,675 Chunks
Type,uint16,numpy.ndarray


In [13]:
tracks_df

,Time (hours),x,y,Area,Intracellular Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID
53400,0,843.888977,9.895291,1423.0,0.005217,0.015092,0.958017,0.000000,WT,PZA,EC99,307,"(5, 6)"
53401,1,842.016602,8.402156,1206.0,0.005233,0.017785,0.959472,2.394837,WT,PZA,EC99,307,"(5, 6)"
53402,2,854.697998,12.947233,2331.0,0.005259,0.015358,0.938589,13.471286,WT,PZA,EC99,307,"(5, 6)"
53403,3,853.580505,11.769628,1745.0,0.005243,0.015071,0.931874,1.623436,WT,PZA,EC99,307,"(5, 6)"
53404,4,853.345703,12.378826,1993.0,0.005525,0.016826,0.929846,0.652882,WT,PZA,EC99,307,"(5, 6)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56320,70,178.384598,1155.093018,4675.0,0.005638,0.035036,0.976493,5.144382,WT,PZA,EC99,158,"(5, 6)"
56321,71,200.807510,1140.815552,3834.0,0.005791,0.046956,0.932750,26.582570,WT,PZA,EC99,158,"(5, 6)"
56322,72,198.752075,1142.213135,3965.0,0.005737,0.040138,0.944715,2.485569,WT,PZA,EC99,158,"(5, 6)"
56323,73,199.767929,1142.548218,3792.0,0.005731,0.039431,0.950185,1.069691,WT,PZA,EC99,158,"(5, 6)"


In [14]:
tracks_df.loc[tracks_df['Intracellular Mtb content'].idxmax()]

Time (hours)                         44
x                            922.763916
y                            375.779785
Area                             1703.0
Intracellular Mtb content        0.0087
Macroph. GFP expression        0.072102
Eccentricity                   0.796489
MSD                            4.293594
Strain                               WT
Compound                            PZA
Concentration                      EC99
Cell ID                             431
Acquisition ID                   (5, 6)
Name: 54569, dtype: object

In [15]:
tracks_df.loc[tracks_df['Intracellular Mtb content'].idxmin()]

Time (hours)                         73
x                              9.147012
y                            980.150269
Area                              619.0
Intracellular Mtb content      0.005074
Macroph. GFP expression        0.021289
Eccentricity                   0.787484
MSD                            3.421689
Strain                               WT
Compound                            PZA
Concentration                      EC99
Cell ID                             193
Acquisition ID                   (5, 6)
Name: 55498, dtype: object

In [16]:
cell_ID_193 = tracks_df[tracks_df['Cell ID']==193]
cell_ID_431 = tracks_df[tracks_df['Cell ID']==431]

In [17]:
cell_ID_431

,Time (hours),x,y,Area,Intracellular Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID
54525,0,927.022827,392.380096,2544.0,0.005816,0.034374,0.841795,0.000000,WT,PZA,EC99,431,"(5, 6)"
54526,1,929.788879,390.037354,2705.0,0.005753,0.029524,0.513802,3.624843,WT,PZA,EC99,431,"(5, 6)"
54527,2,933.411438,386.697540,2202.0,0.005769,0.029265,0.838534,4.927198,WT,PZA,EC99,431,"(5, 6)"
54528,3,936.937927,392.752991,1498.0,0.005817,0.033970,0.826542,7.007468,WT,PZA,EC99,431,"(5, 6)"
54529,4,939.611145,384.385254,1952.0,0.005800,0.031106,0.779231,8.784368,WT,PZA,EC99,431,"(5, 6)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54595,70,894.532715,346.507751,2001.0,0.007900,0.062009,0.604818,4.101465,WT,PZA,EC99,431,"(5, 6)"
54596,71,896.276428,348.002594,2308.0,0.007694,0.060276,0.662960,2.296757,WT,PZA,EC99,431,"(5, 6)"
54597,72,895.013977,346.771790,2147.0,0.007720,0.061512,0.587702,1.763140,WT,PZA,EC99,431,"(5, 6)"
54598,73,895.066040,349.307098,2286.0,0.007723,0.059950,0.680616,2.535843,WT,PZA,EC99,431,"(5, 6)"


### Removing a track from the df because it is broken

In [153]:
tracks_df.drop(tracks_df[tracks_df['Cell ID'] == 114].index, inplace = True)

In [154]:
tracks_df.loc[tracks_df['Intracellular Mtb content'].idxmax()]

Time (hours)                          2
x                            271.020325
y                             95.521255
Area                              541.0
Intracellular Mtb content      0.007181
Macroph. GFP expression        0.045807
Eccentricity                   0.708602
MSD                            5.292714
Strain                               WT
Compound                            PZA
Concentration                      EC99
Cell ID                             329
Acquisition ID                   (5, 6)
Name: 53702, dtype: object

## Select one cell as a seperate df

In [155]:
sc_df = tracks_df[tracks_df['Cell ID']==329]
ID = list(sc_df['Cell ID'])[0]
acq_ID = list(sc_df['Acquisition ID'])[0]

## Make series of glimpses

In [156]:
### glimpse size
size = 500
### resized images
scale = 6048/1200

In [157]:
%%time
glimpse_stack = list()
for row in tqdm(sc_df.iterrows(), total = len(sc_df)):
    time, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
    frame = images[time,...]
    x1, y1 = x*scale, y*scale
#     x1, x2, y1, y2 = x1*scale, x2*scale, y1*scale, y2*scale
#     x1, x2, y1, y2 = x1, x2+size, y1, y2+size
    x1, x2, y1, y2 = x1, x1+size, y1, y1+size
    frame = da.pad(frame, [(0, 0), (size/2, size/2), (size/2, size/2)], 'constant', constant_values = 0) 
    glimpse = frame[..., int(x1): int(x2), int(y1): int(y2)]# frame[..., int(x1): int(x2), int(y1): int(y2)]

    glimpse_stack.append(glimpse)
glimpse_stack = np.stack(glimpse_stack, axis = 1)
# #     fn = 'cell_ID_' + str(cell.ID) + '_channel' + channel + '_t{}_age{}.tif'.format(i, age)
#     ### glimpse output
#     imsave(os.path.join(output_ch_dir, fn), glimpse) 
# print('cell ID',cell.ID, 'glimpse saved')

  0%|          | 0/75 [00:00<?, ?it/s]

CPU times: user 514 ms, sys: 32.4 ms, total: 547 ms
Wall time: 525 ms


In [158]:
%%time
glimpse_stack = glimpse_stack.compute().compute()

CPU times: user 20min 38s, sys: 34.4 s, total: 21min 12s
Wall time: 1min 40s


In [159]:
glimpse_stack.shape

(2, 75, 500, 500)

In [160]:
def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time:1.1f} hours"
text_size = 18
scale = [1.4949402023919043e-07, 1.4949402023919043e-07]

### Check glimpse with labels

In [166]:
viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<function __main__.update_slider(event)>

### Use camera angle for animation/mp4 creation

In [162]:
zoom = viewer.camera.zoom
cam_coords = viewer.camera.center

In [165]:
ID

329

In [164]:
fn = f'/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/{acq_ID}/{ID}/glimpse_{ID}.mp4'
if not os.path.exists(os.path.dirname(fn)):
    os.mkdir(os.path.dirname(fn))

In [167]:

viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

animation = Animation(viewer)
viewer.update_console({'animation': animation})
# viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
viewer.camera.zoom = zoom*0.85
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)

animation.animate(fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
# viewer.close()

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...


100%|█████████████████████████████████████████| 101/101 [00:04<00:00, 24.95it/s]


In [42]:
t = 0

In [45]:
from skimage.io import imsave

In [170]:
os.mkdir(os.path.dirname(fn)+f'/{ID}_glimpse_seq')

FileExistsError: [Errno 17] File exists: '/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/(5, 6)/329/329_glimpse_seq'

In [171]:
viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

for t in tqdm(list(sc_df['Time (hours)'])):
    viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
    viewer.camera.zoom = zoom*0.85
    new_fn = os.path.dirname(fn)+f'/{ID}_glimpse_seq/t_{t}.tiff'
    imsave(new_fn, viewer.screenshot())

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


  0%|          | 0/75 [00:00<?, ?it/s]

# Custom way of plotting animated cells

In [56]:
sc_df

,Time (hours),x,y,Area,Intracellular Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID
56100,0,25.314745,28.294371,1865.0,0.005497,0.017963,0.821394,0.000000,WT,PZA,EC99,146,"(5, 6)"
56101,1,24.051659,19.478317,1568.0,0.005463,0.019028,0.610784,8.906076,WT,PZA,EC99,146,"(5, 6)"
56102,2,23.336201,13.710711,1279.0,0.005088,0.019344,0.761441,5.811812,WT,PZA,EC99,146,"(5, 6)"
56103,3,28.879093,17.372795,1191.0,0.005106,0.015283,0.332211,6.643381,WT,PZA,EC99,146,"(5, 6)"
56104,4,26.880720,16.584396,1333.0,0.005123,0.020810,0.664694,2.148271,WT,PZA,EC99,146,"(5, 6)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56170,70,57.471886,22.544073,1316.0,0.006837,0.027534,0.550928,2.190062,WT,PZA,EC99,146,"(5, 6)"
56171,71,58.136665,22.929312,1061.0,0.007211,0.030545,0.304900,0.768336,WT,PZA,EC99,146,"(5, 6)"
56172,72,57.172527,26.031071,1223.0,0.006932,0.028406,0.496024,3.248149,WT,PZA,EC99,146,"(5, 6)"
56173,73,59.558529,31.047659,1196.0,0.006991,0.028969,0.497672,5.555102,WT,PZA,EC99,146,"(5, 6)"


In [61]:
from scipy.ndimage.filters import gaussian_filter1d


/tmp/ipykernel_135603/1543485559.py:1: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [172]:
os.mkdir(os.path.dirname(new_fn).replace('_seq', f'_animated_graph/'))

In [ ]:
y = list(sc_df['Intracellular Mtb content'].interpolate())
y = gaussian_filter1d(y, sigma=2.5)
for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df)):
#     x = list(cell_ID_431['Time'].iloc[0:n+1])
#     y = list(cell_ID_431['Intracellular Mtb content'].iloc[0:n+1].interpolate())
#     y = gaussian_filter1d(y, sigma=2.5)
    x = list(sc_df['Time (hours)'].iloc[0:n+1])
    
    plot_y = y[0:n+1]

    t = row[1]['Time (hours)'] 
    ID = row[1]['Cell ID']
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,8), gridspec_kw={'width_ratios': [1.15, 1]})
#     fig.suptitle(f'Cell ID: {ID}')
    glimpse_fn = os.path.dirname(new_fn)+f'/t_{t}.tiff'
    glimpse_img = imread(glimpse_fn)
    ax1.imshow(glimpse_img)

    ax1.axis('off')
    ax2.plot(x, plot_y,)# c = palette[1])
    ax2.set_aspect(220/0.0085)
    ax2.set(xlabel = 'Time (hours)', ylabel = f'Mtb expression (rfp intensity) in cell {ID}', ylim=(0.005,0.00725), xlim = (0,75))
    sns.despine()
    new_graph_fn = os.path.dirname(new_fn).replace('_seq', f'_animated_graph/smooth_t_{t}.png')
    plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)

  0%|          | 0/75 [00:00<?, ?it/s]

/tmp/ipykernel_135603/1128066596.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,8), gridspec_kw={'width_ratios': [1.15, 1]})


## Compile into mp4

In [ ]:
import glob
from natsort import natsorted
from napari_animation import Animation

In [ ]:
os.path.dirname(new_graph_fn)

In [ ]:
plots = list()
for fn in tqdm(natsorted(glob.glob(os.path.dirname(new_graph_fn)+'/smooth_t_*.png')), ):#total = 56):
    plot = imread(fn)
    plots.append(plot)
plots = np.stack(plots, axis = 0)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
            #    )
viewer.theme = 'light'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = text_size
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = text_size
# viewer.dims.events.c

In [150]:
cam_coords = viewer.camera.center
zoom = viewer.camera.zoom

In [151]:
final_fn = os.path.dirname(new_graph_fn)+'.mp4'

In [91]:
final_fn

'/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/(5, 6)/146/146_glimpse_animated_graph.mp4'

In [152]:
viewer = napari.Viewer()
viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
            #    )
viewer.theme = 'light'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = text_size
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = text_size
# viewer.dims.events.current_step.connect(update_slider)

animation = Animation(viewer)
viewer.update_console({'animation': animation})
# # viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
viewer.camera.zoom = zoom
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (55.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)

animation.animate(final_fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
viewer.close()

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...


100%|█████████████████████████████████████████| 101/101 [00:04<00:00, 20.75it/s]


# Iterate over many cells

In [9]:
tracks_df.drop(tracks_df[tracks_df['Cell ID'] == 329].index, inplace = True)

In [154]:
tracks_df.loc[tracks_df['Intracellular Mtb content'].idxmax()]

Time (hours)                          2
x                            271.020325
y                             95.521255
Area                              541.0
Intracellular Mtb content      0.007181
Macroph. GFP expression        0.045807
Eccentricity                   0.708602
MSD                            5.292714
Strain                               WT
Compound                            PZA
Concentration                      EC99
Cell ID                             329
Acquisition ID                   (5, 6)
Name: 53702, dtype: object

## Select one cell as a seperate df

In [155]:
sc_df = tracks_df[tracks_df['Cell ID']==329]
ID = list(sc_df['Cell ID'])[0]
acq_ID = list(sc_df['Acquisition ID'])[0]

## Make series of glimpses

In [156]:
### glimpse size
size = 500
### resized images
scale = 6048/1200

In [157]:
%%time
glimpse_stack = list()
for row in tqdm(sc_df.iterrows(), total = len(sc_df)):
    time, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
    frame = images[time,...]
    x1, y1 = x*scale, y*scale
#     x1, x2, y1, y2 = x1*scale, x2*scale, y1*scale, y2*scale
#     x1, x2, y1, y2 = x1, x2+size, y1, y2+size
    x1, x2, y1, y2 = x1, x1+size, y1, y1+size
    frame = da.pad(frame, [(0, 0), (size/2, size/2), (size/2, size/2)], 'constant', constant_values = 0) 
    glimpse = frame[..., int(x1): int(x2), int(y1): int(y2)]# frame[..., int(x1): int(x2), int(y1): int(y2)]

    glimpse_stack.append(glimpse)
glimpse_stack = np.stack(glimpse_stack, axis = 1)
# #     fn = 'cell_ID_' + str(cell.ID) + '_channel' + channel + '_t{}_age{}.tif'.format(i, age)
#     ### glimpse output
#     imsave(os.path.join(output_ch_dir, fn), glimpse) 
# print('cell ID',cell.ID, 'glimpse saved')

  0%|          | 0/75 [00:00<?, ?it/s]

CPU times: user 514 ms, sys: 32.4 ms, total: 547 ms
Wall time: 525 ms


In [158]:
%%time
glimpse_stack = glimpse_stack.compute().compute()

CPU times: user 20min 38s, sys: 34.4 s, total: 21min 12s
Wall time: 1min 40s


In [159]:
glimpse_stack.shape

(2, 75, 500, 500)

In [160]:
def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time:1.1f} hours"
text_size = 18
scale = [1.4949402023919043e-07, 1.4949402023919043e-07]

### Check glimpse with labels

In [166]:
viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<function __main__.update_slider(event)>

### Use camera angle for animation/mp4 creation

In [162]:
zoom = viewer.camera.zoom
cam_coords = viewer.camera.center

In [165]:
ID

329

In [164]:
fn = f'/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/{acq_ID}/{ID}/glimpse_{ID}.mp4'
if not os.path.exists(os.path.dirname(fn)):
    os.mkdir(os.path.dirname(fn))

In [167]:

viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

animation = Animation(viewer)
viewer.update_console({'animation': animation})
# viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
viewer.camera.zoom = zoom*0.85
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)

animation.animate(fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
# viewer.close()

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...


100%|█████████████████████████████████████████| 101/101 [00:04<00:00, 24.95it/s]


In [42]:
t = 0

In [45]:
from skimage.io import imsave

In [170]:
os.mkdir(os.path.dirname(fn)+f'/{ID}_glimpse_seq')

FileExistsError: [Errno 17] File exists: '/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/(5, 6)/329/329_glimpse_seq'

In [171]:
viewer = napari.Viewer()
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
                )
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

for t in tqdm(list(sc_df['Time (hours)'])):
    viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
    viewer.camera.zoom = zoom*0.85
    new_fn = os.path.dirname(fn)+f'/{ID}_glimpse_seq/t_{t}.tiff'
    imsave(new_fn, viewer.screenshot())

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


  0%|          | 0/75 [00:00<?, ?it/s]

# Custom way of plotting animated cells

In [56]:
sc_df

,Time (hours),x,y,Area,Intracellular Mtb content,Macroph. GFP expression,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID
56100,0,25.314745,28.294371,1865.0,0.005497,0.017963,0.821394,0.000000,WT,PZA,EC99,146,"(5, 6)"
56101,1,24.051659,19.478317,1568.0,0.005463,0.019028,0.610784,8.906076,WT,PZA,EC99,146,"(5, 6)"
56102,2,23.336201,13.710711,1279.0,0.005088,0.019344,0.761441,5.811812,WT,PZA,EC99,146,"(5, 6)"
56103,3,28.879093,17.372795,1191.0,0.005106,0.015283,0.332211,6.643381,WT,PZA,EC99,146,"(5, 6)"
56104,4,26.880720,16.584396,1333.0,0.005123,0.020810,0.664694,2.148271,WT,PZA,EC99,146,"(5, 6)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56170,70,57.471886,22.544073,1316.0,0.006837,0.027534,0.550928,2.190062,WT,PZA,EC99,146,"(5, 6)"
56171,71,58.136665,22.929312,1061.0,0.007211,0.030545,0.304900,0.768336,WT,PZA,EC99,146,"(5, 6)"
56172,72,57.172527,26.031071,1223.0,0.006932,0.028406,0.496024,3.248149,WT,PZA,EC99,146,"(5, 6)"
56173,73,59.558529,31.047659,1196.0,0.006991,0.028969,0.497672,5.555102,WT,PZA,EC99,146,"(5, 6)"


In [61]:
from scipy.ndimage.filters import gaussian_filter1d


/tmp/ipykernel_135603/1543485559.py:1: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [172]:
os.mkdir(os.path.dirname(new_fn).replace('_seq', f'_animated_graph/'))

In [ ]:
y = list(sc_df['Intracellular Mtb content'].interpolate())
y = gaussian_filter1d(y, sigma=2.5)
for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df)):
#     x = list(cell_ID_431['Time'].iloc[0:n+1])
#     y = list(cell_ID_431['Intracellular Mtb content'].iloc[0:n+1].interpolate())
#     y = gaussian_filter1d(y, sigma=2.5)
    x = list(sc_df['Time (hours)'].iloc[0:n+1])
    
    plot_y = y[0:n+1]

    t = row[1]['Time (hours)'] 
    ID = row[1]['Cell ID']
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,8), gridspec_kw={'width_ratios': [1.15, 1]})
#     fig.suptitle(f'Cell ID: {ID}')
    glimpse_fn = os.path.dirname(new_fn)+f'/t_{t}.tiff'
    glimpse_img = imread(glimpse_fn)
    ax1.imshow(glimpse_img)

    ax1.axis('off')
    ax2.plot(x, plot_y,)# c = palette[1])
    ax2.set_aspect(220/0.0085)
    ax2.set(xlabel = 'Time (hours)', ylabel = f'Mtb expression (rfp intensity) in cell {ID}', ylim=(0.005,0.00725), xlim = (0,75))
    sns.despine()
    new_graph_fn = os.path.dirname(new_fn).replace('_seq', f'_animated_graph/smooth_t_{t}.png')
    plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)

  0%|          | 0/75 [00:00<?, ?it/s]

/tmp/ipykernel_135603/1128066596.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,8), gridspec_kw={'width_ratios': [1.15, 1]})


## Compile into mp4

In [ ]:
import glob
from natsort import natsorted
from napari_animation import Animation

In [ ]:
os.path.dirname(new_graph_fn)

In [ ]:
plots = list()
for fn in tqdm(natsorted(glob.glob(os.path.dirname(new_graph_fn)+'/smooth_t_*.png')), ):#total = 56):
    plot = imread(fn)
    plots.append(plot)
plots = np.stack(plots, axis = 0)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
            #    )
viewer.theme = 'light'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = text_size
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = text_size
# viewer.dims.events.c

In [150]:
cam_coords = viewer.camera.center
zoom = viewer.camera.zoom

In [151]:
final_fn = os.path.dirname(new_graph_fn)+'.mp4'

In [91]:
final_fn

'/home/dayn/Videos/tb_mp4s/glimpses/pierre_data/(5, 6)/146/146_glimpse_animated_graph.mp4'

In [152]:
viewer = napari.Viewer()
viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
            #    )
viewer.theme = 'light'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = text_size
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = text_size
# viewer.dims.events.current_step.connect(update_slider)

animation = Animation(viewer)
viewer.update_console({'animation': animation})
# # viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
viewer.camera.zoom = zoom
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (55.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)

animation.animate(final_fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
viewer.close()

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...


100%|█████████████████████████████████████████| 101/101 [00:04<00:00, 20.75it/s]
